# Playing with new 2017 `tf.contrib.seq2seq`

I just discovered that a [new dynamic seq2seq implementation](https://github.com/tensorflow/tensorflow/tree/24466c2e6d32621cd85f0a78d47df6eed2c5c5a6/tensorflow/contrib/seq2seq) was recently merged into master. Naturally, I wanted to try it out.

`Working with commit 24466c2e6d32621cd85f0a78d47df6eed2c5c5a6`

In [1]:
import numpy as np
import tensorflow as tf
import helpers

tf.reset_default_graph()
sess = tf.InteractiveSession()

In [2]:
tf.__version__

'1.0.0-alpha'

In [3]:
PAD = 0
EOS = 1

vocab_size = 10
input_embedding_size = 20

encoder_hidden_units = 20
decoder_hidden_units = encoder_hidden_units * 2

Everything is time-major

In [4]:
encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name='encoder_inputs')
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='encoder_inputs_length')

decoder_targets = tf.placeholder(shape=(None, None), dtype=tf.int32, name='decoder_targets')
decoder_targets_length = tf.placeholder(shape=(None,), dtype=tf.int32, name='decoder_targets_length')

During training, `decoder_targets` would serve as basis for both `decoder_inputs` and decoder logits. This means that their shapes should be compatible.

Here we do a bit of plumbing to set this up.

In [5]:
sequence_size, batch_size = tf.unstack(tf.shape(decoder_targets))

EOS_SLICE = tf.ones([1, batch_size], dtype=tf.int32)
PAD_SLICE = tf.zeros([1, batch_size], dtype=tf.int32)

decoder_train_inputs = tf.concat_v2([EOS_SLICE, decoder_targets], axis=0)
decoder_train_length = decoder_targets_length + 1

decoder_train_targets = tf.concat_v2([decoder_targets, PAD_SLICE], axis=0)
decoder_train_targets_seqlen, _ = tf.unstack(tf.shape(decoder_train_targets))
decoder_train_targets_eos_mask = tf.transpose(tf.one_hot(decoder_train_length - 1, decoder_train_targets_seqlen, dtype=tf.int32), [1, 0])
decoder_train_targets = tf.add(
    decoder_train_targets,
    decoder_train_targets_eos_mask,
)  # hacky way to put EOS symbol at the end of target sequence

loss_weights = tf.ones([batch_size, tf.reduce_max(decoder_train_length)], dtype=tf.float32, name="loss_weights")

In [6]:
# oh=tf.one_hot(decoder_train_length, decoder_train_targets_seqlen, dtype=tf.int32)

# sess.run([oh, decoder_train_targets_eos_mask], fd)

In [7]:
from tensorflow.contrib.rnn import (LSTMCell, LSTMStateTuple, EmbeddingWrapper)
import tensorflow.contrib.seq2seq as seq2seq

## Embedding

In [8]:
import math
from tensorflow.contrib.layers import embedding_lookup_unique

def embedding(inputs, embedding_classes, embedding_size, scope=None):
    with tf.variable_scope(scope) as scope:

        # Uniform(-sqrt(3), sqrt(3)) has variance=1.
        sqrt3 = math.sqrt(3)
        initializer = tf.random_uniform_initializer(-sqrt3, sqrt3)
        
        embedding_values = tf.get_variable(
            name="embedding",
            shape=[embedding_classes, embedding_size],
            initializer=initializer,
            dtype=tf.float32)
        
        return embedding_lookup_unique(embedding_values, inputs)

with tf.variable_scope("embedding") as scope:
    encoder_inputs_embedded = embedding(encoder_inputs,
                                        vocab_size,
                                        input_embedding_size,
                                        scope)
    
    scope.reuse_variables()

    decoder_train_inputs_embedded = embedding(decoder_train_inputs,
                                              vocab_size,
                                              input_embedding_size,
                                              scope)
    
    # we'll need matrix for inference
    embedding_matrix = tf.get_variable("embedding")

## Encoder

In [9]:
with tf.variable_scope("Encoder") as scope:
    encoder_cell = LSTMCell(encoder_hidden_units)
    
    ((encoder_fw_outputs,
      encoder_bw_outputs),
     (encoder_fw_state,
      encoder_bw_state)) = (
        tf.nn.bidirectional_dynamic_rnn(cell_fw=encoder_cell,
                                        cell_bw=encoder_cell,
                                        inputs=encoder_inputs_embedded,
                                        sequence_length=encoder_inputs_length,
                                        dtype=tf.float32, time_major=True, scope=scope)
        )

    encoder_outputs = tf.concat_v2((encoder_fw_outputs, encoder_fw_outputs), 2)

    encoder_state_c = tf.concat_v2(
        (encoder_fw_state.c, encoder_bw_state.c), 1)

    encoder_state_h = tf.concat_v2(
        (encoder_fw_state.h, encoder_bw_state.h), 1)

    encoder_state = LSTMStateTuple(
        c=encoder_state_c,
        h=encoder_state_h
    )

## Decoder

#### Training

In [10]:
decoder_cell = LSTMCell(decoder_hidden_units)

def output_fn(outputs, scope):
    return tf.contrib.layers.linear(outputs, vocab_size, scope=scope)

with tf.variable_scope("Decoder") as scope:
    decoder_fn_train = seq2seq.simple_decoder_fn_train(encoder_state=encoder_state)
    
    (
        decoder_outputs_train,
        decoder_state_train,
        decoder_context_state_train
    ) = (
        seq2seq.dynamic_rnn_decoder(
           cell=decoder_cell,
           decoder_fn=decoder_fn_train,
           inputs=decoder_train_inputs_embedded,
           sequence_length=decoder_train_length,
           time_major=True,
           scope=scope,
        )
    )
    
    decoder_logits_train = output_fn(decoder_outputs_train, scope)
    
    decoder_prediction_train = tf.argmax(decoder_logits_train, axis=-1, name='decoder_prediction')


seqloss = seq2seq.sequence_loss(
    logits=tf.transpose(decoder_logits_train, [1, 0, 2]),
    targets=tf.transpose(decoder_train_targets, [1, 0]),
    weights=loss_weights)

loss = tf.reduce_mean(seqloss)
train_op = tf.train.AdamOptimizer().minimize(loss)

/usr/local/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


#### Inference

In [11]:
with tf.variable_scope("Decoder", reuse=True) as scope:
    
    def output_fn_inference(outputs):
        return output_fn(outputs, scope=scope)
    
    decoder_fn_inference = seq2seq.simple_decoder_fn_inference(
        output_fn=output_fn_inference,
        encoder_state=encoder_state, 
        embeddings=embedding_matrix,
        start_of_sequence_id=1,
        end_of_sequence_id=1,
        maximum_length=tf.reduce_max(encoder_inputs_length) + 3,
        num_decoder_symbols=vocab_size,
    )
    
    (
        decoder_outputs_inference,
        decoder_state_inference,
        decoder_context_state_inference
    ) = (
        seq2seq.dynamic_rnn_decoder(
           cell=decoder_cell,
           decoder_fn=decoder_fn_inference,
           time_major=True,
           scope=scope,
        )
    )
    
    decoder_logits_inference = output_fn(decoder_outputs_inference, scope)

ValueError: Trying to share variable Decoder/weights, but specified shape (10, 10) and found shape (40, 10).

## Training

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
import helpers

In [ ]:
batch_size = 100

batches = helpers.random_sequences(length_from=3, length_to=8,
                                   vocab_lower=2, vocab_upper=10,
                                   batch_size=batch_size)

seq = next(batches)[0]
print(seq)

In [ ]:
def next_feed():
    batch = next(batches)
    inputs_, inputs_length_ = helpers.batch(batch)
    return {
        encoder_inputs: inputs_,
        encoder_inputs_length: inputs_length_,
        decoder_targets: inputs_,
        decoder_targets_length: inputs_length_,
    }

In [ ]:
loss_track = []

In [ ]:
for k, v in next_feed().items():
    print(k, len(v))

In [ ]:
max_batches = 10000
batches_in_epoch = 1000

def transpose(l):
    return [x.T for x in l]

try:
    for batch in range(max_batches):
        fd = next_feed()
        _, l = sess.run([train_op, loss], fd)
        loss_track.append(l)

        if batch == 0 or batch % batches_in_epoch == 0:
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            for i, (e_in, d_tg, dt_in, dt_tg, dt_pred) in enumerate(zip(
                    fd[encoder_inputs].T, 
                    fd[decoder_targets].T,
                    *transpose(sess.run([
                        decoder_train_inputs,
                        decoder_train_targets,
                        decoder_prediction_train,
                    ], fd))
                )):
                print('  sample {}:'.format(i + 1))
                print('    enc input           > {}'.format(e_in))
                print('    dec target          > {}'.format(d_tg))
                print('    dec train input     > {}'.format(dt_in))
                print('    dec train target    > {}'.format(dt_tg))
                print('    dec train predicted > {}'.format(dt_pred))
                if i >= 2:
                    break
            print()

except KeyboardInterrupt:
    print('training interrupted')

In [ ]:
dt_eos_mask

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(loss_track)
print('loss {:.4f} after {} examples (batch_size={})'.format(loss_track[-1], len(loss_track)*batch_size, batch_size))

## Attention

In [ ]:
# with tf.variable_scope("Decoder") as scope:
#     attention_states = tf.transpose(encoder_outputs, [1, 0, 2])
    
#     (attention_keys,
#      attention_values,
#      attention_score_fn,
#      attention_construct_fn) = (
#         seq2seq.prepare_attention(
#             attention_states,
#             attention_option="luong", 
#             num_units=decoder_hidden_units)
#     )
    
#     decoder_fn_train = seq2seq.attention_decoder_fn_train(
#               encoder_state=encoder_state,
#               attention_keys=attention_keys,
#               attention_values=attention_values,
#               attention_score_fn=attention_score_fn,
#               attention_construct_fn=attention_construct_fn)
    
#     decoder_cell = LSTMCell(decoder_hidden_units)
#     (decoder_outputs_train, decoder_state_train, _) = (
#           seq2seq.dynamic_rnn_decoder(
#               cell=decoder_cell,
#               decoder_fn=decoder_fn_train,
#               inputs=decoder_inputs,
#               sequence_length=decoder_length,
#               time_major=True,
#               scope=scope))